Before we begin, let's execute the cell below to display information about the CUDA driver and GPUs running on the server by running the `nvidia-smi` command. To do this, execute the cell block below by giving it focus (clicking on it with your mouse), and hitting Ctrl-Enter, or pressing the play button in the toolbar above. If all goes well, you should see some output returned below the grey cell.

In [ ]:
!nvidia-smi

## Learning objectives
The **goal** of this lab is to:

- Learn how to find bottleneck and performance limiters using Nsight tools
- Learn about ways to to extract more parallelism in loops


In this section, we will optimize the parallel [RDF](../serial/rdf_overview.ipynb) application using OpenMP offloading. Before we begin, feel free to have a look at the parallel version of the code and inspect it once again. 

[RDF Parallel Code](../../source_code/openmp/SOLUTION/rdf_offload.cpp)

[File Reader](../../source_code/openmp/SOLUTION/dcdread.h)

Now, let's compile, and profile it with Nsight Systems first.

In [ ]:
#compile for Tesla GPU
!cd ../../source_code/openmp && nvc++ -mp=gpu -Minfo=mp -o rdf SOLUTION/rdf_offload.cpp

In [ ]:
#profile and see output of nvptx
!cd ../../source_code/openmp && nsys profile -t nvtx,cuda --stats=true --force-overwrite true -o rdf_offload ./rdf

Let's checkout the profiler's report. Download and save the report file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](../../source_code/openacc/rdf_offload.qdrep) and open it via the GUI.

## Further Optimization
Currently both our distributed and workshared parallelism comes from the same loop. In other words, in this example, we used teams and distribute to spread parallelism across the full GPU. As a result, all of the parallelism comes from the outer loop. There are multiple ways to increase parallelism: 

- Moving the `Parallel For` to the inner loop
- Collapsing the loops together

In the following sections, we will explore each of these methods.


### Split Teams Distribute from Parallel For

In this method, we distribute outer loop to thread teams and workshare inner loop across threads.The example below shows usage of OpenMP target splitting teams distribute from parallel for. The example code contains two levels of OpenMP offloading parallelism in a double-nested loop.

```cpp
#pragma omp target teams distribute 
    for( int j = 1; j < N; j++) {
      #pragma parallel for 
      for( int i = 1; i < M; i++ ) {
            <loop code>
    }
}
```

Look at the profiler report from the previous section again. From the timeline, have a close look at the the kernel functions. Checkout the theoretical [occupancy](../GPU_Architecture_Terminologies.ipynb). As shown in the example screenshot below, `pair_gpu_182` kernel has the theoretical occupancy of 50%. It clearly shows that the occupancy is a limiting factor. *Occupancy* is a measure of how well the GPU compute resources are being utilized. It is about how much parallelism is running / how much parallelism the hardware could run.

<img src="../images/data_thread.png">

NVIDIA GPUs are comprised of multiple [streaming multiprocessors (SMs)](../GPU_Architecture_Terminologies.ipynb) where it can manage up to 2048 concurrent threads (not actively running at the same time). Low occupancy shows that there are not enough active [threads](../GPU_Architecture_Terminologies.ipynb) to fully utilize the computing resources. Higher occupancy implies that the scheduler has more active threads to choose from and hence achieves higher performance. So, what does this mean in OpenACC execution model?



Now, lets start modifying the original code and split the teams distribute. From the top menu, click on *File*, and *Open* `rdf.cpp` and `dcdread.h` from the current directory at `C/source_code/openmp` directory. Remember to **SAVE** your code after changes, before running below cells.

In [ ]:
#compile for Tesla GPU
!cd ../../source_code/openmp && nvc++ -mp=gpu -Minfo=mp -o rdf rdf.cpp 

Inspect the compiler feedback (you should get a similar output as below) you can see from *Line 172* and *Line 178* that it separated the teams distribute from parallel for.

<img src="../images/openmp_feedback_offload_split.png">

Now, validate the output by running the executable, and then **Profile** your code with Nsight Systems command line `nsys`.

In [ ]:
#Run on Nvidia GPU and check the output
!cd ../../source_code/openmp && ./rdf && cat Pair_entropy.dat

The output should be the following:

```
s2 value is -2.43191
s2bond value is -3.87014
```

In [ ]:
#profile and see output of nvptx
!cd ../../source_code/openmp && nsys profile -t nvtx,cuda --stats=true --force-overwrite true -o rdf_offload_split ./rdf

Let's checkout the profiler's report. Download and save the report file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](../../source_code/openmp/rdf_offload_split.qdrep) and open it via the GUI. Compare the execution time for the `Pair_Calculation` from the NVTX row with the previous section and see how much the performance was improved.

Feel free to checkout the [solution](../../source_code/openmp/SOLUTION/rdf_offload_split.cpp) to help you understand better.

## Post-Lab Summary

If you would like to download this lab for later viewing, it is recommend you go to your browsers File menu (not the Jupyter notebook file menu) and save the complete web page.  This will ensure the images are copied down as well. You can also execute the following cell block to create a zip-file of the files you've been working on, and download it with the link below.

In [ ]:
%%bash
cd ..
rm -f nways_files.zip
zip -r nways_files.zip *

**After** executing the above zip command, you should be able to download and save the zip file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](../nways_files.zip).
Let us now go back to parallelizing our code using other approaches.

**IMPORTANT**: If you wish to dig deeper and profile the kernel with the Nsight Compute profiler, go to the next notebook. Otherwise, please click on **HOME** to go back to the main notebook for *N ways of GPU programming for MD* code. 

-----

# <p style="text-align:center;border:3px; border-style:solid; border-color:#FF0000  ; padding: 1em"> <a href=../../../nways_MD_start.ipynb>HOME</a>&nbsp; &nbsp; &nbsp; <a href=nways_openmp_opt_2.ipynb>NEXT</a></p>

-----



# Links and Resources
[OpenMP Programming Model](https://computing.llnl.gov/tutorials/openMP/)

[OpenMP Target Directive](https://www.openmp.org/wp-content/uploads/openmp-examples-4.5.0.pdf)

[NVIDIA Nsight System](https://docs.nvidia.com/nsight-systems/)

[NVIDIA Nsight Compute](https://developer.nvidia.com/nsight-compute)

**NOTE**: To be able to see the Nsight System profiler output, please download Nsight System latest version from [here](https://developer.nvidia.com/nsight-systems).

Don't forget to check out additional [OpenACC Resources](https://www.openacc.org/resources) and join our [OpenACC Slack Channel](https://www.openacc.org/community#slack) to share your experience and get more help from the community.

--- 

## Licensing 

This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0).